# **BACK-END**

> Advanced Natural Language Processing

> Muhammad Sameer (XNV7SX)

> Muhammad Umar (044014)

> University of Debrecen










**Install Requirements for Back-End**

In [ ]:
!pip install -q llama-index \
    llama-index-embeddings-huggingface \
    llama-index-llms-huggingface \
    bitsandbytes \
    kagglehub

Executing on Cloud...

**Backend Setup**

In [4]:
import os
import pandas as pd
import kagglehub
from tqdm import tqdm

from llama_index.core import Document, VectorStoreIndex, StorageContext, load_index_from_storage, Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.memory import ChatMemoryBuffer

f:\Advanced Natural Language Processing\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**Load dataset**

In [3]:
path = kagglehub.dataset_download(handle="googleai/dataset-metadata-for-cord19")
filename = os.path.join(path, os.listdir(path)[0])
df = pd.read_csv(filename)
df = df[df['description'].notnull()][['description']]


**Chunk text into 150-word sections**

In [4]:
chunks = []
chunk_size = 150
for text in tqdm(df['description']):
    words = str(text).split()
    for i in range(0, len(words), chunk_size):
        chunk = " ".join(words[i:i + chunk_size])
        chunks.append(Document(text=chunk))


100%|██████████| 14126/14126 [00:00<00:00, 29566.24it/s]


**Set embedding model**

In [ ]:
Settings.embed_model = HuggingFaceEmbedding(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    device="cuda"
)

**Load or create vector index**

In [6]:
persist_dir = "storage"
if not os.path.exists(persist_dir):
    print("🔧 Creating new index...")
    index = VectorStoreIndex.from_documents(chunks, show_progress=True)
    index.storage_context.persist(persist_dir=persist_dir)
else:
    print("📂 Loading index from disk...")
    storage = StorageContext.from_defaults(persist_dir=persist_dir)
    index = load_index_from_storage(storage)

🔧 Creating new index...


Parsing nodes:   0%|          | 0/20433 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

...

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2005 [00:00<?, ?it/s]

**Use falcon-7b**

In [ ]:
from llama_index.llms.huggingface import HuggingFaceLLM

Settings.llm = HuggingFaceLLM(
    model_name="tiiuae/falcon-7b-instruct",
    tokenizer_name="tiiuae/falcon-7b-instruct",
    context_window=2048,
    max_new_tokens=256,
    device_map="auto",
    tokenizer_kwargs={"use_fast": True},
    model_kwargs={
        "load_in_4bit": True,
        "bnb_4bit_quant_type": "nf4",
        "bnb_4bit_use_double_quant": True,
        "trust_remote_code": True
    },
    generate_kwargs={"temperature": 0.7, "do_sample": True},
)


**Create chat engine**

In [8]:
chat_engine = index.as_chat_engine(
    chat_mode="context",
    memory=ChatMemoryBuffer.from_defaults(token_limit=2048),
    system_prompt="""
You are MedAI, a medical chatbot. Answer only the user's question.
Do not repeat it. Do not ask follow-up questions. Be concise and factual.
"""
)


_________________________________________________________________________________________________________________________


____________________________________________________________________________

# **Front-END**

**Install Requirements for Front-End**

In [ ]:
!pip install gradio llama-index llama-index-embeddings-huggingface llama-index-llms-huggingface bitsandbytes --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 113.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.5 MB/s eta 0:00:00


In [10]:
import gradio as gr

**Function to handle chat**

In [11]:
def medai_chat(user_input, _):
    try:
        user_input = user_input.strip()
        response = chat_engine.chat(user_input)
        return response.response
    except Exception as e:
        return f"[ERROR] {str(e)}"

**Gradio interface**

In [12]:
chat_ui = gr.ChatInterface(
    fn=medai_chat,
    chatbot=gr.Chatbot(label="🩺 MedAI Assistant"),
    title="MedAI - COVID & Smoking Research Chatbot",
    description="Ask specific questions about COVID-19, smoking, and respiratory health. MedAI will respond concisely and factually.",
    theme="soft",
    examples=[
        "What are the risks of smoking during COVID-19?",
        "Can COVID-19 worsen lung conditions?",
        "How does smoking affect immune response?",
         "What are the similar effects on patients who smoke and those with COVID-19 in the human body?",
        "Can smoking increase the severity of COVID-19 symptoms?",
        "What organs are affected by smoking and COVID-19?"
    ]
)

chat_ui.launch(share=True)

<ipython-input-12-e1b15360c1ee>:3: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot=gr.Chatbot(label="🩺 MedAI Assistant"),
/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:321: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://80aa32f71d05d0018a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
